In [1]:
import numpy as np
import tensorflow as tf
import random

2023-10-05 16:47:43.367894: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-05 16:47:44.353528: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [37]:
class Grid():
    def __init__(self): 
        self.rows = 10
        self.cols = 10
        
        self.goal = tuple(np.random.randint((self.rows, self.cols)))
        self.block = tuple(np.random.randint((self.rows, self.cols)))
        
        self.agent_list = []
        self.agent_positions = []

        self.action_shape = 10
        self.state_list = []
        
        self.model = self._centralized_network()
        
    def add_agent(self, agent, grid):
        agent.id = self.num_agents
        agent.position = self.spawn_agent()
        agent.model = self.model
        agent.grid = grid
        self.agent_positions.append(agent.position)
        self.agent_list.append(agent)

    def spawn_agent(self):
        while True:
            position = tuple(np.random.randint((self.rows, self.cols)))
            if position not in self.agent_positions and position != self.goal and position != self.block:
                return position
                
    @property
    def num_agents(self):
        return len(self.agent_list)

    def global_state(self):
        state = np.zeros((self.rows, self.cols))
        for agent in self.agent_list:
            state[agent.position[0]][agent.position[1]] = 1 #agent.id + 1
        state[self.goal[0]][self.goal[1]] = -1
        state[self.block[0]][self.block[1]]= -2
        return state
        
    def _centralized_network(self):
        state_shape = np.shape(self.global_state())
        y = x = tf.keras.layers.Input(state_shape)
        y = tf.keras.layers.Dense(64)(y)
        y = tf.keras.layers.Dense(self.action_shape)(y)
        model = tf.keras.Model(x, y)
        return model

In [38]:
class Action():
    push = 0
    move = 1
    right = [0, 1]
    left = [0, -1]
    up = [-1, 0]
    down = [1, 0]
    no_op = [0, 0]
    
    def __init__(self, type, action):
        self.action_type = type
        self.action = action

    @classmethod
    def random(cls):
        action_type = random.choice([Action.push, Action.move])
        action = random.choice([Action.left, Action.right, Action.up, Action.down])
        return cls(action_type, action)

In [41]:
class Agent():
    def __init__(self):
        self.id = 0
        self.position = 0
        self.epsilon = 0.1
        
        self.grid = []
        
        self.model = []
        
        self.current_state = []
        self.previous_state = []
        
    def __repr__(self):
        return f"Agent(id={self.id}, pos={self.position})"

    def move(self, action, grid):
        if action.action_type == 0:
            grid.block += np.array(action.action)
            grid.block %= [grid.rows, grid.cols]
        
        if action.action_type == 1:
            self.position += np.array(action.action)
            self.position %= [grid.rows, grid.cols]
            
    def current_state():
        print(grid.global_state())
    
    def policy(self):        
        #Get the current state 
        current_state = self.get_state()
        
        #Take random action with some probability
        if random.random() < self.epsilon:
            return Action.random()
        #Otherwise pick the best action to take
        else:
            predicted_action_q_values = self.model.predict(current_state)
            best_action_q_value = np.max(predicted_action_q_values)
            action = Actions[best_action_q_value]

In [42]:
grid = Grid()

In [43]:
for i in range(5):
    agent = Agent()
    grid.add_agent(agent, grid)

In [44]:
grid.global_state()

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0., -2.,  1.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [46]:
grid.agent_list

[Agent(id=0, pos=(7, 0)),
 Agent(id=1, pos=(6, 0)),
 Agent(id=2, pos=(4, 6)),
 Agent(id=3, pos=(4, 9)),
 Agent(id=4, pos=(2, 5))]

In [ ]:
action = Action()

In [54]:
grid.agent_list[0].move()

TypeError: Agent.move() missing 2 required positional arguments: 'action' and 'grid'